In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

%matplotlib inline

In [2]:
camry = pd.read_csv('Camry_242_Spring2023.csv')
camry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MonthNumeric   174 non-null    int64  
 1   MonthFactor    174 non-null    object 
 2   Year           174 non-null    int64  
 3   CamrySales     174 non-null    int64  
 4   Unemployment   174 non-null    float64
 5   CamryQueries   174 non-null    int64  
 6   CPIAll         174 non-null    float64
 7   CPIEnergy      174 non-null    float64
 8   MilesTraveled  174 non-null    int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 12.4+ KB


In [3]:
camry_train = camry[camry['Year'] <= 2018]
camry_testA = camry[(camry['Year'] >= 2019) & (camry['Year'] <= 2020)]
camry_testB = camry[camry['Year'] >= 2021]

len(camry_train), len(camry_testA), len(camry_testB)

(132, 24, 18)

In [8]:
camry_train = camry[(camry['Year'] == 2018) & (camry['MonthNumeric'] == 1)]

camry_train.head(1)

,MonthNumeric,MonthFactor,Year,CamrySales,Unemployment,CamryQueries,CPIAll,CPIEnergy,MilesTraveled
120,1,January,2018,24638,4.0,81,248.743,216.872,268361


In [4]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['Unemployment', 'CamryQueries', 'CPIAll', 'CPIEnergy', 'MilesTraveled']
X_train = camry_train[cols]
y_train = camry_train['CamrySales']

# We must add an intercept as the standard model doesn't automatically fit one
X_train = sm.add_constant(X_train)

# Fit the data to the model
model1 = sm.OLS(y_train, X_train).fit() #ordinary least square
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     4.722
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000549
Time:                        22:28:31   Log-Likelihood:                -1339.1
No. Observations:                 132   AIC:                             2690.
Df Residuals:                     126   BIC:                             2707.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1.056e+05   5.21e+04      2.026

In [5]:
import matplotlib.pyplot as plt
def coefplot(results):
    '''
    Takes in results of OLS model and returns a plot of 
    the coefficients with 95% confidence intervals.
    
    Removes intercept, so if uncentered will return error.
    '''
    # Create dataframe of results summary 
    coef_df = pd.DataFrame(results.summary().tables[1].data)
    
    # Add column names
    coef_df.columns = coef_df.iloc[0]

    # Drop the extra row with column labels
    coef_df=coef_df.drop(0)

    # Set index to variable names 
    coef_df = coef_df.set_index(coef_df.columns[0])

    # Change datatype from object to float
    coef_df = coef_df.astype(float)

    # Get errors; (coef - lower bound of conf interval)
    errors = coef_df['coef'] - coef_df['[0.025']
    
    # Append errors column to dataframe
    coef_df['errors'] = errors

    # Drop the constant for plotting
    coef_df = coef_df.drop(['const'])

    # Sort values by coef ascending
    coef_df = coef_df.sort_values(by=['coef'])

    ### Plot Coefficients ###

    # x-labels
    variables = list(coef_df.index.values)
    
    # Add variables column to dataframe
    coef_df['variables'] = variables
    
    # Set sns plot style back to 'poster'
    # This will make bars wide on plot
    sns.set_context("poster")

    # Define figure, axes, and plot
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Error bars for 95% confidence interval
    # Can increase capsize to add whiskers
    coef_df.plot(x='variables', y='coef', kind='bar',
                 ax=ax, color='none', fontsize=22, 
                 ecolor='steelblue',capsize=0,
                 yerr='errors', legend=False)
    
    # Set title & labels
    plt.title('Coefficients of Features w/ 95% Confidence Intervals',fontsize=30)
    ax.set_ylabel('Coefficients',fontsize=22)
    ax.set_xlabel('',fontsize=22)
    
    # Coefficients
    ax.scatter(x=np.arange(coef_df.shape[0]), 
               marker='o', s=80, 
               y=coef_df['coef'], color='steelblue')
    
    # Line to define zero on the y-axis
    ax.axhline(y=0, linestyle='--', color='red', linewidth=1)
    
    return plt.show()

In [6]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CamryQueries + CPIAll + CPIEnergy + MilesTraveled', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     4.722
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000549
Time:                        22:28:31   Log-Likelihood:                -1339.1
No. Observations:                 132   AIC:                             2690.
Df Residuals:                     126   BIC:                             2707.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.056e+05   5.21e+04      2.026

In [7]:
# compute out-of-sample R-squared using the test set
def OSR2(model, df_train, df_test, dependent_var):   
    y_test = df_test[dependent_var]
    y_pred = model.predict(df_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(df_train[dependent_var]))**2)    
    return 1 - SSE/SST

In [8]:
print(OSR2(model1, camry_train, camry_testA, 'CamrySales'))
print(OSR2(model1, camry_train, camry_testB, 'CamrySales'))

0.5366480526882766
0.29932807687232243


In [9]:
# calculate Variance Inflation Factor for each explanatory variable
from statsmodels.stats.outliers_influence import variance_inflation_factor

# The dataframe passed to VIF must include the intercept term. We add it the same way we did before.
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

cols = ['Unemployment', 'CamryQueries', 'CPIAll', 'CPIEnergy', 'MilesTraveled']
VIF(camry_train, cols)

Unemployment      4.140307
CamryQueries      6.955649
CPIAll            7.425610
CPIEnergy         2.467946
MilesTraveled    12.610832
dtype: float64

In [10]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CamryQueries + CPIAll + CPIEnergy', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     5.852
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000235
Time:                        22:28:31   Log-Likelihood:                -1339.3
No. Observations:                 132   AIC:                             2689.
Df Residuals:                     127   BIC:                             2703.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     7.902e+04   2.32e+04      3.400   

In [11]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CamryQueries + CPIAll + MilesTraveled', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     5.125
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000734
Time:                        22:28:31   Log-Likelihood:                -1340.6
No. Observations:                 132   AIC:                             2691.
Df Residuals:                     127   BIC:                             2706.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.678e+05   3.69e+04      4.547

In [12]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CamryQueries + CPIEnergy + MilesTraveled', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     5.057
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000817
Time:                        22:28:31   Log-Likelihood:                -1340.7
No. Observations:                 132   AIC:                             2691.
Df Residuals:                     127   BIC:                             2706.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       1.12e+05   5.24e+04      2.137

In [13]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CPIAll + CPIEnergy + MilesTraveled', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     5.512
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000400
Time:                        22:28:32   Log-Likelihood:                -1339.9
No. Observations:                 132   AIC:                             2690.
Df Residuals:                     127   BIC:                             2704.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      7.253e+04   4.45e+04      1.629

In [14]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ CamryQueries + CPIAll + CPIEnergy + MilesTraveled', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     3.367
Date:                Mon, 30 Jan 2023   Prob (F-statistic):             0.0118
Time:                        22:28:32   Log-Likelihood:                -1343.8
No. Observations:                 132   AIC:                             2698.
Df Residuals:                     127   BIC:                             2712.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6979.3620   4.21e+04      0.166

In [15]:
import statsmodels.formula.api as smf

ols = smf.ols(formula='CamrySales ~ Unemployment + CPIAll + CPIEnergy', 
                 data=camry_train)
model1 =ols.fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:             CamrySales   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     7.394
Date:                Mon, 30 Jan 2023   Prob (F-statistic):           0.000131
Time:                        22:30:32   Log-Likelihood:                -1339.9
No. Observations:                 132   AIC:                             2688.
Df Residuals:                     128   BIC:                             2699.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     6.516e+04   1.95e+04      3.345   